In [68]:
/** What a pain in the ass of a question
 *
 *  Here's the idea of what to do:
 *  1. We need to get the length of each column
 *  - We can do this by using the last row which contains the operator. We can count the number of whitespace inbetween each operator
 *  - Cavet: The last operator doesn't have trailing whitespace, so we have to figure it out using the longest data row
 *
 *  2. Then using the length of each column, we have to parse the data row.
 *  - Cavet: The last column doesn't have trailing whitespace, so we can just manually pad it with empty space
 *
 *  3. We're going to deliberately keep the whitespace in the val input. We'll let part1 and part2 handle the whitespace
 *  - Cavet: Whitespace could be in front or behind
 */
val input = File("6.txt").readLines().let { lines ->
    val operatorRow = lines.last()
    val dataRows = lines.dropLast(1)

    val operatorMatches = Regex("\\S+").findAll(operatorRow).toList()
    val numCols = operatorMatches.size

    val columnLengths = (0 until numCols).map { col ->
        if (col < numCols - 1) {
            operatorMatches[col + 1].range.first - operatorMatches[col].range.first - 1
        } else {
            val start = operatorMatches[col].range.first
            val maxDataLength = dataRows.maxOf { row ->
                if (start < row.length) {
                    val remaining = row.substring(start)
                    val numberMatch = Regex("^\\s*\\d+").find(remaining)
                    numberMatch?.value?.length ?: 0
                } else 0
            }
            maxDataLength
        }
    }

    val totalWidth = operatorMatches.last().range.first + columnLengths.last()
    val paddedRows = dataRows.map { it.padEnd(totalWidth, ' ') }

    (0 until numCols).map { col ->
        val start = operatorMatches[col].range.first
        val length = columnLengths[col]

        val numbers = paddedRows.map { row ->
            row.substring(start, start + length)
        }

        val operator = operatorRow.substring(start, start + 1)
        numbers + operator
    }
}

In [70]:
 val part1 = input.sumOf { problems ->
    val numbers = problems.dropLast(1).map { it.trim().toLong() }
    val operation = problems.last()
    if (operation == "+") {
        numbers.sum()
    } else {
        numbers.fold(1) { acc, i -> acc * i }
    }
}

val part2 = input.sumOf { problems ->
    val numbers = problems.dropLast(1)
    val operation = problems.last()

    val maxLength = numbers.maxOf { it.length }
    val vertical = (0 until maxLength).map { col ->
        numbers.map { row ->
            val index = row.length - 1 - col
            if (index >= 0) row[index] else ' '
        }.joinToString("").trim()
    }.filter { it.isNotEmpty() }.map {it -> it.toLong()}

    if (operation == "+") {
        vertical.sum()
    } else {
        vertical.fold(1) { acc, i -> acc * i }
    }
}